# R03 Ví dụ

## Mục đích

Luyện tập sử dụng RegEx với một số ví dụ từ cơ bản đến nâng cao.

## Phát hiện kí tự lặp lại

Vì một lí do nào đó, có thể bạn không muốn trong chuỗi kí tự của bạn có những kí tự lặp lại đứng cạnh nhau. Với RegEx, bạn có thể tìm ra rất dễ dàng.

In [1]:
import re

strings = ["1a8f892l", "dshoiaal", "4s399ahrox"]
pat = r"([a-zA-z0-9])\1"

for s in strings:
    print(f"{s=:12}: {re.search(pat, s) is not None}")

s=1a8f892l    : False
s=dshoiaal    : True
s=4s399ahrox  : True


Cách chúng ta vừa làm là tạo ra một nhóm tóm bắt có một kí tự, kí tự đó có thể là chữ in thường, in hoa, hoặc chữ số. Sau đó, cho nhóm tóm bắt đó lặp lại thêm một lần nữa bằng chuỗi `\1`. Lưu ý rằng cách làm này khác với `[a-zA-z0-9]+`; chuỗi RegEx thứ hai sẽ match bất kì 2 kí tự nào thỏa mãn điều kiện là "chữ in thường, chữ hoa, hoặc chữ số".

## Khớp số điện thoại

Số điện thoại ở Mỹ có thể được viết ở các định dạng như sau:

* 410-356-8888
* (410)356-8888
* 410 356 8888
* 4103568888
* 1 410 356 8888

Bên cạnh việc kiểm tra xem một chuỗi kí tự có đúng là số điện thoại ở Mỹ không, chúng ta sẽ tách phần mã vùng là 3 chữ số đầu tiên ra.

In [2]:
strings = ["410-356-8888", "(410)356-8888", "410 356 8888", "4103568888", "1 410 356 8888"]
pat = r"(?:1\s)?\(?(\d{3})\)?[-\s]?\d{3}[-\s]?\d{4}"

for s in strings:
    m = re.search(pat, s)
    print(f"{s=:12}: {m is not None} --> {'' if m is None else m.group(1)}")

s=410-356-8888: True --> 410
s=(410)356-8888: True --> 410
s=410 356 8888: True --> 410
s=4103568888  : True --> 410
s=1 410 356 8888: True --> 410


Chuỗi RegEx trên đã dài hơn và trông phức tạp hơn, nhưng nó vẫn tương đối đơn giản. Để khớp được tất cả các định dạng chuỗi số điện thoại nói trên, chúng ta cần phải tìm điểm chung:

* Có thể có mã quốc gia (`1`) sau đó là một dấu cách
* Tiếp sau là 3 chữ số mã vùng, có thể bao bởi cặp dấu ngoặc đơn
* Tiếp theo là 3 chữ số và 4 chữ số của số điện thoại, có thể phân cách với nhau bằng dấu gạch ngang (`-`), dấu cách, hoặc viết liền với nhau

Sau khi đã tìm ra đặc điểm chung như vậy, chắc bạn đã hiểu cách chúng ta viết chuỗi RegEx. Với những chỗ "có thể", chúng ta dùng kí tự `?` để khớp các trường hợp có 0 hoặc 1 lần xuất hiện. Với những chỗ có chính xác bao nhiêu lần xuất hiện, chúng ta dùng chuỗi `{m}`.

## Chuỗi mật khẩu

Bạn có thể đặt ra yêu cầu cho một chuỗi mật khẩu, chẳng hạn như: 10 kí tự chữ hoặc số không lặp lại, có từ 2 chữ viết hoa trở lên, và có từ 3 chữ số trở lên. Ví dụ, một chuỗi như sau sẽ được match: `"A123Bcdefg"`.

In [3]:
strings = ["A123Bcdefg", "AA123Bcdef", "A123bcdefg", "A12bcdefgh"]
pat = r"(?!.*(.).*\1)(?=(?:.*[A-Z]){2,})(?=(?:.*\d){3,})[a-zA-Z0-9]{10}"

for s in strings:
    print(f"{s=:12}: {re.search(pat, s) is not None}")

s=A123Bcdefg  : True
s=AA123Bcdef  : False
s=A123bcdefg  : False
s=A12bcdefgh  : False


Với chuỗi RegEx phức tạp như thế này, chúng ta sẽ cần phân tích rất kĩ.

Đầu tiên, bạn sẽ cần khớp điều kiện "10 kí tự chữ hoặc số không lặp lại". Bạn có thể dễ dàng đạt được điều kiện đó với chuỗi: `[a-zA-Z0-9]{10}`.

Tiếp theo, để không có kí tự nào lặp lại, chúng ta sẽ làm hai bước:

* "Kí tự lặp lại" với chuỗi `.*(.).*\1`. Chuỗi này có nghĩa là "bỏ qua 0 hoặc bất kì kí tự nào đầu tiên, sau đó match một kí tự khác với dấu xuống dòng, và bỏ qua 0 hoặc bất kì kí tự nào sau đó, cho đến khi match tiếp kí tự ban đầu lần thứ hai". Ví dụ: với chuỗi `"aa12hc1"`, chúng ta match tuần tự như sau:

    + Match chữ cái `a` đầu tiên. Sau đó chúng ta sẽ match được chữ cái `a` ngay tiếp theo.
    + Match chữ cái `a` thứ hai, nhưng sau đó không còn chữ cái `a` nào nữa. Vậy không match.
    + Match chữ số `1` thứ ba. Sau đó chúng ta bỏ qua các kí tự tiếp theo mà không phải là `1`, cho đến khi match được chữ số `1` cuối cùng.
    + Tiếp tục như vậy...

* "Không có kí tự nào lặp lại": chúng ta sẽ sử dụng negative lookahead `(?!)`.

Đối với hai điều kiện còn lại, chúng ta sẽ sử dụng positive lookahead `(?=)`. Hai điều kiện này có cách viết cú pháp giống nhau. Đầu tiên chúng ta match một chữ hoa bất kì bằng non-capturing group `(?:.*[A-Z])`, sau đó thêm số lượng `{2,}` và cho tất cả vào trong positive lookahead.

## Số thẻ tín dụng

Số thẻ tín dụng cũng cần được validate, chẳng hạn: phải có 16 chữ số, nếu cách nhau bằng dấu gạch ngang `-` thì phải chia làm nhóm 4 chữ số. Chữ số đầu tiên phải là 4, 5, hoặc 6. Không thể lặp lại từ 4 chữ số liên tiếp trở lên.

In [4]:
strings = [
    "4998-5820-3247-5060",
    "4998582032475060",
    "4998-58203247-5060",
    "4998-5820-32475-060",
    "1998-5820-3247-5060",
    "4998-5820-3247-5555",
    "4998-5820-3247-506a"
]
pat = r"(?!.*(\d)-?(\1-?){3,}.*)(([4-6][\d]{15})|([4-6][\d]{3}(-\d{4}){3}))"

for s in strings:
    print(f"{s=:20}: {re.match(pat, s) is not None}")

s=4998-5820-3247-5060 : True
s=4998582032475060    : True
s=4998-58203247-5060  : False
s=4998-5820-32475-060 : False
s=1998-5820-3247-5060 : False
s=4998-5820-3247-5555 : False
s=4998-5820-3247-506a : False


Tương tư như ví dụ 3, chúng ta sẽ chia nhỏ các điều kiện validation để phân tích.

Đầu tiên, chuỗi `([4-6][\d]{15})` đảm bảo 16 kí tự thỏa mãn điều kiện bắt đầu với chữ số 4 đến 6 và 15 kí tự số tiếp theo. Còn chuỗi `([4-6][\d]{3}(-\d{4}){3}))` đảm bảo 16 kí tự thỏa mãn điều kiện bắt đầu với chữ số 4 đến 6, sau đó là 3 chữ số, và 3 lần lặp lại cụm (dấu gạch ngang và 4 chữ số).

Sau đó, chúng ta sẽ cần xử lí nốt điều kiện cuối cùng là không có kí tự số nào lặp lại từ 4 lần liên tiếp trở lên. Các bạn có thể nhớ rằng nếu điều kiện là "không có", chúng ta có thể dùng negative lookahead. Chuỗi `(?!.*(\d)-?(\1-?){3,}.*)` có nghĩa là "match một chữ số bất kì, match tiếp nếu chính chữ số đó xuất hiện lại từ 3 lần trở lên (tổng cộng là 4 lần trở lên), và không match toàn bộ chuỗi kí tự này nếu điều kiện trên xảy ra ở bất kì vị trí nào trong chuỗi kí tự".

---

Hi vọng với các ví dụ từ đơn giản đến nâng cao trên đây, bạn đã hiểu hơn về regular expression và thấy được khả năng của nó trong pattern matching. Mình cũng xin tiết lộ là hầu hết các ví dụ mình lấy trong bài không phải là do mình nghĩ ra mà từ các trang dạy về RegEx, nhưng rất tiếc đã quá lâu rồi mình không ghi lại đường link, nên không chia sẻ lại cho các bạn được. Bạn có thể học thêm từ [RegexOne](https://regexone.com/lesson/introduction_abcs) nhé.

---

[Bài trước](./02_group.ipynb) - [Danh sách bài](../README.md) - [Bài sau](./04_diemthi.ipynb)